In [ ]:
Space X Falcon 9 First Stage Landing Prediction

In [1]:
!pip3 install beautifulsoup4
!pip3 install requests


In [2]:
import sys

import requests
from bs4 import BeautifulSoup
import re
import unicodedata
import pandas as pd

def date_time(table_cells):
    """
    This function returns the data and time from the HTML  table cell
    Input: the  element of a table data cell extracts extra row
    """
    return [data_time.strip() for data_time in list(table_cells.strings)][0:2]

def booster_version(table_cells):
    """
    This function returns the booster version from the HTML  table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=''.join([booster_version for i,booster_version in enumerate( table_cells.strings) if i%2==0][0:-1])
    return out

def landing_status(table_cells):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=[i for i in table_cells.strings][0]
    return out


def get_mass(table_cells):
    mass=unicodedata.normalize("NFKD", table_cells.text).strip()
    if mass:
        mass.find("kg")
        new_mass=mass[0:mass.find("kg")+2]
    else:
        new_mass=0
    return new_mass


def extract_column_from_header(row):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    if (row.br):
        row.br.extract()
    if row.a:
        row.a.extract()
    if row.sup:
        row.sup.extract()
        
    colunm_name = ' '.join(row.contents)
    
    # Filter the digit and empty names
    if not(colunm_name.strip().isdigit()):
        colunm_name = colunm_name.strip()
        return colunm_name    
    
    
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

#TASK 1: Request the Falcon9 Launch Wiki page from its URL
# Step 1: Request the Falcon9 Launch Wiki page
static_url = 'https://en.wikipedia.org/wiki/Falcon_9'  # URL for Falcon 9 launch page
response = requests.get(static_url)

# Use BeautifulSoup() to create a BeautifulSoup object from a response text content
soup = BeautifulSoup(response.text, 'html.parser')

# Use soup.title attribute
print(soup.title)

#TASK 2: Extract all column/variable names from the HTML table header

# Use the find_all function in the BeautifulSoup object, with element type `table`
# Assign the result to a list called `html_tables`
html_tables = soup.find_all('table')

# Let's print the third table and check its content
first_launch_table = html_tables[2]
print(first_launch_table)

# Step 5: Define the extract_column_from_header() function
def extract_column_from_header(th_element):
    # Extract and clean the text from <th> element
    return th_element.get_text(strip=True)

# Step 6: Find all <th> elements in the table header
header_cells = first_launch_table.find_all('th')

# Step 7: Extract column names
column_names = []

for th in header_cells:
    column_name = extract_column_from_header(th)
    if column_name and len(column_name) > 0:
        column_names.append(column_name)

# Step 8: Print the extracted column names
print(column_names)

#TASK 3: Create a data frame by parsing the launch HTML tables
launch_dict= dict.fromkeys(column_names)

# Remove an irrelvant column
# Check if 'Date and time ( )' exists in the launch_dict before deleting
if 'Date and time ( )' in launch_dict:
    del launch_dict['Date and time ( )']

# Let's initialize the launch_dict with each value to be an empty list
launch_dict['Flight No.'] = []
launch_dict['Launch site'] = []
launch_dict['Payload'] = []
launch_dict['Payload mass'] = []
launch_dict['Orbit'] = []
launch_dict['Customer'] = []
launch_dict['Launch outcome'] = []
# Added some new columns
launch_dict['Version Booster'] = []
launch_dict['Booster landing'] = []
launch_dict['Date'] = []
launch_dict['Time'] = []



extracted_row = 0
#Extract each table 
for table_number,table in enumerate(soup.find_all('table',"wikitable plainrowheaders collapsible")):
   # get table row 
    for rows in table.find_all("tr"):
        #check to see if first table heading is as number corresponding to launch a number 
        if rows.th:
            if rows.th.string:
                flight_number=rows.th.string.strip()
                flag=flight_number.isdigit()
        else:
            flag=False
        #get table element 
        row=rows.find_all('td')
        #if it is number save cells in a dictonary 
        if flag:
            extracted_row += 1
            # Flight Number value
            # TODO: Append the flight_number into launch_dict with key `Flight No.`
            #print(flight_number)
            datatimelist=date_time(row[0])
            
            # Date value
            # TODO: Append the date into launch_dict with key `Date`
            date = datatimelist[0].strip(',')
            #print(date)
            
            # Time value
            # TODO: Append the time into launch_dict with key `Time`
            time = datatimelist[1]
            #print(time)
              
            # Booster version
            # TODO: Append the bv into launch_dict with key `Version Booster`
            bv=booster_version(row[1])
            if not(bv):
                bv=row[1].a.string
            print(bv)
            
            # Launch Site
            # TODO: Append the bv into launch_dict with key `Launch Site`
            launch_site = row[2].a.string
            #print(launch_site)
            
            # Payload
            # TODO: Append the payload into launch_dict with key `Payload`
            payload = row[3].a.string
            #print(payload)
            
            # Payload Mass
            # TODO: Append the payload_mass into launch_dict with key `Payload mass`
            payload_mass = get_mass(row[4])
            #print(payload)
            
            # Orbit
            # TODO: Append the orbit into launch_dict with key `Orbit`
            orbit = row[5].a.string
            #print(orbit)
            
            # Customer
            # TODO: Append the customer into launch_dict with key `Customer`
            customer = row[6].a.string
            #print(customer)
            
            # Launch outcome
            # TODO: Append the launch_outcome into launch_dict with key `Launch outcome`
            launch_outcome = list(row[7].strings)[0]
            #print(launch_outcome)
            
            # Booster landing
            # TODO: Append the launch_outcome into launch_dict with key `Booster landing`
            booster_landing = landing_status(row[8])
            #print(booster_landing)
        
        
 #launch record values into launch_dict, you can create a dataframe from it.
df= pd.DataFrame({ key:pd.Series(value) for key, value in launch_dict.items() })

df.to_csv('spacex_web_scraped.csv', index=False)

print(df.head())


<title>Falcon 9 - Wikipedia</title>
<table class="wikitable">
<tbody><tr>
<td>Height
</td>
<td>41.2 m / 135.2 ft
</td></tr>
<tr>
<td>Height (with interstage)
</td>
<td>47.7 m / 156.5 ft
</td></tr>
<tr>
<td>Diameter
</td>
<td>3.7 m / 12 ft
</td></tr>
<tr>
<td>Empty Mass
</td>
<td>25,600 kg / 56,423 lb
</td></tr>
<tr>
<td>Propellant Mass
</td>
<td>395,700 kg/ 872,369 lb
</td></tr>
<tr>
<td>Structure Type
</td>
<td>LOX tank: <a href="/wiki/Monocoque" title="Monocoque">monocoque</a>
</td></tr>
<tr>
<td>
</td>
<td>Fuel tank: skin and stringer
</td></tr>
<tr>
<td>Structure Material
</td>
<td>Aluminum lithium skin; aluminum domes
</td></tr>
<tr>
<td>Landing Legs
</td>
<td>Number: 4
</td></tr>
<tr>
<td>
</td>
<td>Material: <a class="mw-redirect" href="/wiki/Carbon_fiber" title="Carbon fiber">carbon fiber</a>; <a class="mw-redirect" href="/wiki/Aluminum_honeycomb" title="Aluminum honeycomb">aluminum honeycomb</a>
</td></tr>
<tr>
<td>Number of Merlin Engines
</td>
<td>9 sea level
</td></tr>
<tr>

/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/ipykernel_launcher.py:205: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
